In [3]:
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plta

Mounted at /content/drive


In [4]:
import os
os.chdir('drive/MyDrive/')

In [5]:
import os
import numpy as np
import cv2
from google.cloud import storage
from tensorflow.keras.utils import image_dataset_from_directory


In [6]:
directory = 'r_data'
training_directory = f'{directory}/Training'
testing_directory = f'{directory}/Testing'

In [7]:
train_ds, val_ds = image_dataset_from_directory(
  training_directory,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255, 255),
  validation_split = 0.2,
  subset="both",
  color_mode = 'rgb',
  batch_size= 64)

test_ds = image_dataset_from_directory(
  testing_directory,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255, 255),
  color_mode = 'rgb',
  batch_size= 64)

Found 5712 files belonging to 4 classes.
Using 4570 files for training.
Using 1142 files for validation.
Found 1311 files belonging to 4 classes.


In [8]:
class_names = train_ds.class_names
class_names

['glioma', 'meningioma', 'notumor', 'pituitary']

In [9]:
from tensorflow.keras.applications import EfficientNetV2B3
from tensorflow.keras import layers, Sequential


In [10]:

base_model = EfficientNetV2B3(include_top=False,
                       weights='imagenet',
                       input_shape=(255,255,3),
                       pooling='max',
                       include_preprocessing=True) 

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
# model.add(layers.Dropout(0.3))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(4, activation='softmax'))

base_model.trainable = False

model.summary()

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

52606240/52606240 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b3 (Function  (None, 1536)             12930622  
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 64)                98368     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 13,029,250
Trainable params: 98,628
Non-tra

In [11]:
from tensorflow.keras import callbacks

es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

model.fit(train_ds,
          validation_data=val_ds,
          batch_size=64, 
          epochs=100,
          callbacks=[es],
          verbose=1)

Epoch 1/100
72/72 [==============================] - 966s 12s/step - loss: 1.1634 - accuracy: 0.5652 - val_loss: 0.5380 - val_accuracy: 0.8170
Epoch 2/100
72/72 [==============================] - 23s 298ms/step - loss: 0.7042 - accuracy: 0.7107 - val_loss: 0.4687 - val_accuracy: 0.8196
Epoch 3/100
72/72 [==============================] - 21s 278ms/step - loss: 0.5644 - accuracy: 0.7753 - val_loss: 0.3681 - val_accuracy: 0.8538
Epoch 4/100
72/72 [==============================] - 22s 292ms/step - loss: 0.5110 - accuracy: 0.7989 - val_loss: 0.3319 - val_accuracy: 0.8905
Epoch 5/100
72/72 [==============================] - 23s 298ms/step - loss: 0.4786 - accuracy: 0.8149 - val_loss: 0.3497 - val_accuracy: 0.8485
Epoch 6/100
72/72 [==============================] - 23s 300ms/step - loss: 0.4382 - accuracy: 0.8306 - val_loss: 0.2823 - val_accuracy: 0.8993
Epoch 7/100
72/72 [==============================] - 21s 281ms/step - loss: 0.4106 - accuracy: 0.8335 - val_loss: 0.2640 - val_accuracy: 

In [12]:
model.evaluate(test_ds)

21/21 [==============================] - 274s 8s/step - loss: 0.2102 - accuracy: 0.9199


[0.2102469801902771, 0.9199084639549255]

In [13]:
model.save('models/model_Efficientnet.h5')